In [12]:
import pandas as pd
from datetime import datetime
import numpy as np

In [2]:
# Giving column names according to our wish, and reading the excel file
user_cols = ['Invoice_number', 'Date', 'Customer_name', 'Place', 'Product', 'Quantity','Serial','Amount','Tax','Contact_number']
orders = pd.read_excel('JUNE  2017-18.xls', names = user_cols, skiprows = 5, header = None)
orders.head()

,Invoice_number,Date,Customer_name,Place,Product,Quantity,Serial,Amount,Tax,Contact_number
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3090,05.06.17,MM Forgings,Guindy,Inter link cables,1No,NaN,NaN,NaN,22352518
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3092,05.06.17,MM Forgings,Guindy,100 Ah B Rack,1No,NaN,NaN,NaN,9840980468
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# dropping the columns Serial, Amount and Tax as almost all entries are of 'NaN'

orders.dropna(how = 'all', inplace = True)
orders.drop(orders.columns[[6, 7, 8]], axis = 1, inplace = True)
orders.head()

,Invoice_number,Date,Customer_name,Place,Product,Quantity,Contact_number
1,3090,05.06.17,MM Forgings,Guindy,Inter link cables,1No,22352518
3,3092,05.06.17,MM Forgings,Guindy,100 Ah B Rack,1No,9840980468
5,3093,05.06.17,Valli Textile,Virudhunagar,PSDR Board,1No,04562-284615/28
7,3099,07.06.17,Tafe Access,Kelambakkam,APFC Panel,1No,27498145
9,3101,09.06.17,Sysveda Information,Perungudi,Inter Link Cable,1No,9841076572


In [ ]:
orders.info()

In [5]:
orders.describe()

,Invoice_number,Date,Customer_name,Place,Product,Quantity,Contact_number
count,95,95,101,102,150,143,79
unique,95,25,78,68,99,37,64
top,2157,30.06.17,MM Forgings,Guindy,Eaton 9145 1 UPS,1No,9940915004
freq,1,14,8,8,12,51,4


The above description says : 

   * Most active day of the month : ** 30th June 2017, 14 items. **
   * Most active client of the month : ** MM Forgings, 8 times **
   * Maximum orders came from : ** Guindy, Chennai , 8 times **
   * Best selling product of the month : ** Eaton 9145 1 UPS, 12 times **
   * Total Sales of the month : ** 95 Invoices generated **

In [8]:
# for mailing purpose taking the essential fields alone
essentials = orders[['Date', 'Invoice_number', 'Customer_name', 'Contact_number']].copy()
# .copy() to avoid SettingWithCopyWarning
essentials.dropna(how = 'any',inplace = True)
# drops the row even if one of the field is missing 
essentials.sort_values('Date', inplace = True)
# Sorts the date string
essentials.head()

,Date,Invoice_number,Customer_name,Contact_number
44,01.06.17,2093,USUI Susira,9710013151
40,01.06.17,2092,USUI Susira,9710013151
36,01.06.17,2091,USUI Susira,9710013151
51,02.06.17,2095,Hueco India,9600073851
57,03.06.17,2098,Park Plaza,9382244150


In [9]:
essentials.shape

(76, 4)

In [ ]:
# for eliminating telephone numbers where messages can't be sent
rejected = []
for number in essentials['Contact_number']:
    if len(str(number)) != 10:
        rejected.append(number)
rejected

In [ ]:
# Converting every element in the df into string dtype
essentials.applymap(str)

In [14]:
# for getting the contact details of the customer_name along with the date of purchase and invoice number
# also resetting the index into 1, 2, 3 ...
essentials.index = np.arange(1, len(essentials)+1)
essentials.head()

,Date,Invoice_number,Customer_name,Contact_number
1,01.06.17,2093,USUI Susira,9710013151
2,01.06.17,2092,USUI Susira,9710013151
3,01.06.17,2091,USUI Susira,9710013151
4,02.06.17,2095,Hueco India,9600073851
5,03.06.17,2098,Park Plaza,9382244150


In [15]:
# exporting the above dataframe into a csv file named 'resq-June-2017-phone-numbers.csv'
essentials.to_csv('resq-June-2017.csv', index = True,  index_label = 'Serial', encoding='utf-8')

In [ ]:
# for getting mobile numbers alone, which are of 10 digits
essentials = essentials[essentials['Contact_number'].map(lambda x:len(str(x))) == 10]

In [ ]:
essentials.reset_index(drop=True, inplace = True)
essentials

In [ ]:
essentials.to_csv('resq-June-2017.csv', index = True,  index_label = 'Serial', encoding='utf-8')

In [26]:
df = pd.read_csv('resq-June-2017.csv', index_col = 0)
df.head()

,Date,Invoice_number,Customer_name,Contact_number
Serial,,,,
1,01.06.17,2093,USUI Susira,9710013151
2,01.06.17,2092,USUI Susira,9710013151
3,01.06.17,2091,USUI Susira,9710013151
4,02.06.17,2095,Hueco India,9600073851
5,03.06.17,2098,Park Plaza,9382244150


In [27]:
# to get the one year old dates from the csv and send email to the admin, to renew the client's product 

current_date = datetime.now().strftime("%d.%m.%y")
to_check = current_date[:-2]
df = df.loc[df['Date'] == to_check + str(int(current_date[-2:]) - 1), :]

if df.empty:
    msg = 'No renewals today'
else:
    df.index = np.arange(1,len(df)+1)
    contact_clients = []
    for index, row in df.iterrows():
         contact_clients.append(str(index) + ' ' + str(row['Date']) + ' ' + str(row['Date']) + ' ' + str(row['Customer_name']) + ' ' + str(row['Contact_number']))

In [20]:
# in case of no orders on the same date last year.
msg

'No renewals today'

In [28]:
# had to manually change all 29.06.17 to 29.07.16 to prove how would the program react, if renewal is supposed to happen this is how it would behave else above scenario. 
contact_clients

['1 29.07.16 29.07.16 Sundaram Clayton Ltd 9840967320',
 '2 29.07.16 29.07.16 Vivekanada Institute of Bio Technology 9474129451',
 '3 29.07.16 29.07.16 Konica Colour Lab 9841416519',
 '4 29.07.16 29.07.16 Medopharm 9551016009',
 '5 29.07.16 29.07.16 Swaminathan Enterprie 7550097670']

In [ ]:
# for sending the email to admin of resq
# note : had to 'allow permission to less secure apps', in gmail else you would get 'SMTPAuthenticationError', though your email id and password being correct

import smtplib
smtpObj = smtplib.SMTP('smtp.gmail.com', 587)
smtpObj.ehlo()
smtpObj.starttls()
smtpObj.login("jaiimmortal@gmail.com", "NoWayThisIsMyPwd6969")
if df.empty:
    msg = 'No renewals today'
    smtpObj.sendmail("jaiimmortal@gmail.com", "yoyaswant@gmail.com", 'Subject: {}\n'.format(current_date[:-3]) + msg)
else:
    s = '\n'
    msg = s.join(contact_clients)
    smtpObj.sendmail("jaiimmortal@gmail.com", "yoyaswant@gmail.com", 'Subject: {}\n'.format(current_date[:-3]) + msg)
smtpObj.quit()
